In [28]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_z


In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pprint import pprint
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

class selfintroduce(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미")
    goal: str = Field(description="미래 목표")



parser = PydanticOutputParser(pydantic_object=selfintroduce)


template = """
학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미,미래 목표를 selfintroduce와 같이 구조화된 형태로 추출하세요.
자기소개: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.0,
    model_kwargs={
        "response_format": { "type": "json_object" }
    }
)


query = """
안녕하세요! 저는 김민수이고 22살입니다. 
컴퓨터공학을 전공하고 있어요. 
취미로는 게임하기, 영화보기, 코딩을 좋아합니다.
 앞으로 훌륭한 개발자가 되는 것이 목표입니다.
"""
chain = prompt | model | parser
output = chain.invoke({"query": query})

print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미: {', '.join(output.hobbies)}")
print(f"미래 목표: {output.goal}")

gsk_z
이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
미래 목표: 훌륭한 개발자가 되는 것
